# Outcome Model Calibration

Train a probabilistic model to replace the heuristic catch/incomplete/interception probabilities.

## Load Dataset

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, classification_report
from sklearn.preprocessing import StandardScaler
import joblib

REPO_ROOT = Path('..').resolve().parents[0]
data_path = REPO_ROOT / 'analytics' / 'data' / 'outcome_training.parquet'
print('Dataset path:', data_path)
data = pd.read_parquet(data_path)
data.head()

## Feature Engineering

In [ ]:
FEATURES = [
    'dacs_final','dacs_final_lo','dacs_final_hi','coverage_intensity','dvi','bfoi',
    'prob_catch','prob_incomplete','prob_interception',
    'n_defenders','corridor_length','num_frames_output'
]
X = data[FEATURES].fillna(0.0).to_numpy(dtype=np.float32)
y = data['actual_event_id'].to_numpy(dtype=np.int64)
folds = data['fold_id'].to_numpy(dtype=np.int64)
X_train, y_train = X[folds != 0], y[folds != 0]
X_val, y_val = X[folds == 0], y[folds == 0]
scaler = StandardScaler().fit(X_train)
X_train_s = scaler.transform(X_train)
X_val_s = scaler.transform(X_val)
print('Train size', X_train.shape, 'Val size', X_val.shape)

## Multinomial Logistic Regression

In [ ]:
clf = LogisticRegression(max_iter=1000, multi_class='multinomial')
clf.fit(X_train_s, y_train)
train_loss = log_loss(y_train, clf.predict_proba(X_train_s))
val_loss = log_loss(y_val, clf.predict_proba(X_val_s))
print({'train_log_loss': train_loss, 'val_log_loss': val_loss})
print(classification_report(y_val, clf.predict(X_val_s)))

## Persist Model

In [ ]:
model_path = REPO_ROOT / 'analytics' / 'models' / 'outcome_model.joblib'
model_path.parent.mkdir(parents=True, exist_ok=True)
joblib.dump({'scaler': scaler, 'clf': clf, 'features': FEATURES}, model_path)
print('Saved model to', model_path)

## Calibration Plots
Add reliability diagrams per class to verify probability quality.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve

classes = ['catch','incomplete','interception']
val_proba = clf.predict_proba(X_val_s)
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
for idx, ax in enumerate(axes):
    prob_true, prob_pred = calibration_curve((y_val == idx).astype(int), val_proba[:, idx], n_bins=10)
    ax.plot(prob_pred, prob_true, marker='o')
    ax.plot([0,1],[0,1], '--', color='gray')
    ax.set_title(f'Reliability: {classes[idx]}')
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Observed')
plt.tight_layout()
plt.show()